## Simply Convert Data to COCO Format

- Split the training data to train and validation
- Convert both to COCO Formatted JSON

In [ ]:
# Some basic setup:
# import some common libraries
import numpy as np
import pandas as pd
import os
import json
import random
import cv2
import matplotlib.pyplot as plt
import ast
from tqdm import tqdm
from sklearn.model_selection import train_test_split

In [ ]:
TOP_INPUT_DIR = "/kaggle/input/global-wheat-detection/"
DATA_TRAIN_DIR = f"{TOP_INPUT_DIR}train/"
DATA_TEST_DIR = f"{TOP_INPUT_DIR}test/"

### Create COCO Base

In [ ]:
coco_base = { "info": {},
              "licenses": [], 
              "images": [],
              "annotations": [],
              "categories": []}

### Write dataset info to COCO Format 

In [ ]:
coco_base["info"] = {
    "description": "Global Wheat Detection Dataset, Kaggle 2020",
    "url": "https://www.kaggle.com/c/global-wheat-detection/data",
    "version": "1.0",
    "year": 2020,
    "contributor": "http://www.global-wheat.com/contributors/",
    "date_created": "2020/05/29"
}

### Write licenses to COCO Format

In [ ]:
coco_base["licenses"].append(
    {
        "url": "https://opensource.org/licenses/MIT",
        "id": 1,
        "name": "MIT License"
    }
)

### Write category to COCO Format

In [ ]:
coco_base["categories"].append({"supercategory": "grain","id": 1,"name": "wheat"})

In [ ]:
train_df = pd.read_csv(f'{TOP_INPUT_DIR}train.csv')
train_df.head()

In [ ]:
uniq_images = train_df.image_id.unique()
len(uniq_images)

#### There are 3373 unique images in the training set

In [ ]:
print(train_df.width.unique()[0])
print(train_df.height.unique()[0])

#### All images are of size 1024x1024

### Group bboxes by image

Create one row per image - combining all bboxes for that image into one column. This helps significantly reduce the number of rows to traverse.

In [ ]:
def get_bboxes_per_image(df):
    """author: @impiyush"""
    # convert from string list, to python list
    df.bbox = df.bbox.apply(ast.literal_eval)
    # group-by on image id and return all bboxes for that image as a list of lists
    return train_df.groupby('image_id')['bbox'].apply(list).reset_index(name='bboxes')

In [ ]:
train_df_bboxes_grped = get_bboxes_per_image(train_df)
train_df_bboxes_grped.head()

In [ ]:
assert train_df_bboxes_grped.shape[0] == len(uniq_images), "Number of images differ when grouped"

### Split to train and validation

In [ ]:
df_train, df_valid = train_test_split(
    train_df_bboxes_grped,
    test_size=0.05,
    random_state=32,
    shuffle=True
)

df_train = df_train.reset_index(drop=True)
df_valid = df_valid.reset_index(drop=True)

In [ ]:
print(df_train.shape[0], df_valid.shape[0])

### Write images to COCO Format

In [ ]:
coco_base_train = coco_base.copy()
coco_base_valid = coco_base.copy()

In [ ]:
def set_coco_images(df):
    """author: @impiyush"""
    images = []
    for _,img in enumerate(tqdm(df.image_id.unique())):
        img_dict = {"license":1, 
                    "height":1024, 
                    "width":1024,
                    "id":img}
        img_dict["file_name"] = f"{img}.jpg"
        images.append(img_dict)

    return images

In [ ]:
coco_base_train["images"] = set_coco_images(df_train)
coco_base_valid["images"] = set_coco_images(df_valid)

Let's check the first three images in the list

In [ ]:
coco_base_train["images"][:3]

In [ ]:
coco_base_valid["images"][:3]

Let's also verify the number of images match, just for sanity

In [ ]:
assert len(coco_base_train["images"])==len(df_train), "Number of images differ from df_train"
assert len(coco_base_valid["images"])==len(df_valid), "Number of images differ from df_valid"

### Write annotations to COCO Format

The most important piece of this puzzle

In [ ]:
def set_coco_annotations(df):
    """author: @impiyush"""
    annos = []
    id_cnt = 1

    for _,row in tqdm(df.iterrows(), total=len(df)):
    #     print(row)
        anno = {}
        anno['segmentation'] = []
        anno['iscrowd'] = 0
        anno['image_id'] = row['image_id']
        anno['category_id'] = 1
        bboxes = row['bboxes']
    #     annos.append(box)
        for ix, box in enumerate(bboxes):
            anno['bbox'] = box # x,y,w,h
            anno['area'] = box[2] * box[3] # w*h
            anno['id'] = f"{id_cnt:05}"
            annos.append(anno.copy()) # copy is necessary here, otherwise it will always point to the last value of anno
            id_cnt += 1
    
    return annos

In [ ]:
coco_base_train['annotations'] = set_coco_annotations(df_train)
coco_base_valid['annotations'] = set_coco_annotations(df_valid)

### Dump COCO formatted JSON for train and validation sets

In [ ]:
with open(f'/kaggle/working/coco_train.json','w') as train_coco:
    json.dump(coco_base_train, train_coco)

In [ ]:
with open(f'/kaggle/working/coco_valid.json','w') as valid_coco:
    json.dump(coco_base_valid, valid_coco)

### There you go, now you have your data in COCO Format. Hope you liked this quick Kernel!